# RunnableBranch: 입력에 따라 동적으로 로직 라우팅하기

`RunnableBranch` 는 입력에 따라 동적으로 로직을 라우팅할 수 있는 강력한 도구입니다. 이를 통해 개발자는 **입력 데이터의 특성에 기반하여 다양한 처리 경로를 유연하게 정의** 할 수 있습니다.

`RunnableBranch` 는 복잡한 의사 결정 트리를 간단하고 직관적인 방식으로 구현할 수 있도록 도와줍니다. 이는 코드의 가독성과 유지보수성을 크게 향상시키며, 로직의 모듈화와 재사용성을 촉진합니다.

또한, `RunnableBranch` 는 런타임에 동적으로 분기 조건을 평가하고 적절한 처리 루틴을 선택할 수 있어, 시스템의 적응력과 확장성을 높여줍니다.

이러한 특징들로 인해 RunnableBranch는 다양한 도메인에서 활용될 수 있으며, 특히 입력 데이터의 다양성과 변동성이 큰 애플리케이션 개발에 매우 유용합니다. `RunnableBranch` 를 효과적으로 활용하면 코드의 복잡성을 줄이고, 시스템의 유연성과 성능을 향상시킬 수 있습니다.


## 입력에 따른 동적 로직 라우팅

LangChain Expression Language에서 라우팅을 수행하는 방법에 대해 다룹니다.

라우팅을 통해 이전 단계의 출력이 다음 단계를 정의하는 비결정적 체인을 생성할 수 있습니다. 라우팅은 LLM과의 상호 작용에 구조와 일관성을 제공하는 데 도움이 됩니다.

라우팅을 수행하는 방법에는 두 가지가 있습니다.

1. `RunnableLambda` 에서 조건부로 실행 가능한 객체를 반환 (권장)
2. `RunnableBranch` 사용

두 가지 방법 모두 첫 번째 단계에서 입력 질문을 `LangChain`, `Anthropic` 또는 `Other`에 대한 것으로 분류한 다음, 해당 프롬프트 체인으로 라우팅하는 두 단계 시퀀스를 사용하여 설명하겠습니다.


## 간단한 예시

먼저, 들어오는 질문이 `수학`, `과학`, 또는 `기타` 중 하나로 분류하는 Chain을 생성하겠습니다.


In [16]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """주어진 사용자 질문을 `수학`, `과학`, 또는 `기타` 중 하나로 분류하세요. 한 단어 이상으로 응답하지 마세요.

<question>
{question}
</question>

Classification:"""
)

# 체인을 생성합니다.
chain = (
    prompt
    | ChatOpenAI(model="gpt-4-turbo-preview")
    | StrOutputParser()  # 문자열 출력 파서를 사용합니다.
)

생성한 chain을 사용하여 질문을 분류합니다.


In [17]:
# 질문을 입력하여 체인을 호출합니다.
chain.invoke({"question": "2+2 는 무엇인가요?"})

'수학'

In [18]:
# 질문을 입력하여 체인을 호출합니다.
chain.invoke({"question": "작용 반작용의 법칙은 무엇인가요?"})

'과학'

In [19]:
# 질문을 입력하여 체인을 호출합니다.
chain.invoke({"question": "Google은 어떤 회사인가요?"})

'기타'

이제 세 개의 하위 체인을 생성해 보겠습니다.


In [27]:
math_chain = (
    PromptTemplate.from_template(
        """You are an expert in math. \
Always answer questions starting with "깨봉선생님께서 말씀하시기를..". \
Respond to the following question:

Question: {question}
Answer:"""
    )
    # OpenAI의 LLM을 사용합니다.
    | ChatOpenAI(model="gpt-4-turbo-preview")
)

science_chain = (
    PromptTemplate.from_template(
        """You are an expert in science. \
Always answer questions starting with "아이작 뉴턴 선생님께서 말씀하시기를..". \
Respond to the following question:

Question: {question}
Answer:"""
    )
    # OpenAI의 LLM을 사용합니다.
    | ChatOpenAI(model="gpt-4-turbo-preview")
)

general_chain = (
    PromptTemplate.from_template(
        """Respond to the following question concisely:

Question: {question}
Answer:"""
    )
    # OpenAI의 LLM을 사용합니다.
    | ChatOpenAI(model="gpt-4-turbo-preview")
)

## 사용자 정의 함수 사용하기 (권장)

서로 다른 출력 간의 라우팅을 위해 **사용자 정의 함수**를 활용할 수도 있습니다.

다음은 그 예시입니다.


In [28]:
def route(info):
    # 주제에 "수학"이 포함되어 있는 경우
    if "수학" in info["topic"].lower():
        # datascience_chain을 반환
        return math_chain
    # 주제에 "과학"이 포함되어 있는 경우
    elif "과학" in info["topic"].lower():
        # art_chain을 반환
        return science_chain
    # 그 외의 경우
    else:
        # general_chain을 반환
        return general_chain

In [32]:
from langchain_core.runnables import RunnableLambda

full_chain = (
    {"topic": chain, "question": lambda x: x["question"]}
    | RunnableLambda(
        # 경로를 지정하는 함수를 인자로 전달합니다.
        route
    )
    | StrOutputParser()
)

In [33]:
# 수학과 관련된 질문을 입력하여 체인을 호출합니다.
full_chain.invoke({"question": "미적분의 개념에 대해 말씀해 주세요."})

'깨봉선생님께서 말씀하시기를, 미적분은 크게 두 가지 주요 개념인 미분과 적분으로 나뉩니다. 미분은 변화율을 다루는 것으로, 어떤 함수가 주어졌을 때 그 함수의 순간적인 변화율을 계산하는 과정입니다. 예를 들어, 속도는 거리에 대한 시간의 변화율로 볼 수 있습니다. 반면, 적분은 주어진 구간에서 함수의 전체 변화량을 구하는 것으로, 미분과는 반대의 과정입니다. 적분을 통해 넓이나 부피 같은 것들을 계산할 수 있습니다. 간단히 말해, 미적분은 변화하는 세상을 수학적으로 이해하는 강력한 도구입니다.'

In [34]:
# 과학과 관련된 질문을 입력하여 체인을 호출합니다.
full_chain.invoke({"question": "중력은 어떻게 작용하나요?"})

'아이작 뉴턴 선생님께서 말씀하시기를, 중력은 모든 질량을 가진 물체들 사이에 작용하는 힘이며, 이 힘은 두 물체의 질량에 비례하고 두 물체 사이의 거리의 제곱에 반비례합니다. 즉, 물체가 더 무겁거나 서로 가까울수록 중력의 힘이 강해집니다. 이 원리는 뉴턴의 만유인력 법칙으로 알려져 있으며, 지구에서 사과가 떨어지는 것부터 달이 지구 주변을 도는 것, 심지어는 태양계의 행성들이 태양 주변을 도는 것까지 모든 중력 현상을 설명합니다.'

In [35]:
# 기타 질문을 입력하여 체인을 호출합니다.
full_chain.invoke({"question": "RAG(Retrieval Augmented Generation)은 무엇인가요?"})

'RAG(Retrieval Augmented Generation)는 인공지능 분야에서 사용되는 기술로, 질문에 대한 답변을 생성하기 전에 관련 정보를 검색하여 찾아온 후, 그 정보를 바탕으로 답변을 생성하는 방식입니다. 기존의 단순한 문답 생성 방식보다 더 정확하고 풍부한 정보를 제공할 수 있습니다.'

## RunnableBranch 사용하기

`RunnableBranch`는 입력값에 따라 실행할 조건과 Runnable을 정의할 수 있는 특별한 유형의 Runnable입니다.

다만, 위에서 설명한 사용자 정의 함수로 구현할 수 없는 기능을 제공하지는 않으므로, 사용자 정의 함수를 사용하는 것이 좋습니다.

**문법**

- `RunnableBranch`는 (조건, Runnable) 쌍의 리스트와 기본 Runnable로 초기화됩니다.
- 호출 시 전달된 입력값을 각 조건에 전달하여 분기를 선택합니다.
- True로 평가되는 첫 번째 조건을 선택하고, 해당 조건에 해당하는 Runnable을 입력값과 함께 실행합니다.
- 제공된 조건과 일치하는 것이 없으면 기본 Runnable을 실행합니다.


In [38]:
from langchain_core.runnables import RunnableBranch

branch = RunnableBranch(
    # 주제에 "수학"이 포함되어 있는지 확인하고, 포함되어 있다면 math_chain을 실행합니다.
    (lambda x: "수학" in x["topic"].lower(), math_chain),
    # 주제에 "과학"이 포함되어 있는지 확인하고, 포함되어 있다면 science_chain을 실행합니다.
    (lambda x: "과학" in x["topic"].lower(), science_chain),
    # 위의 조건에 해당하지 않는 경우 general_chain을 실행합니다.
    general_chain,
)
# 주제와 질문을 입력받아 branch를 실행하는 전체 체인을 정의합니다.
full_chain = {"topic": chain,
              "question": lambda x: x["question"]} | branch | StrOutputParser()

In [39]:
# 질문을 입력하여 전체 체인을 실행합니다.
full_chain.invoke({"question": "미적분의 개념에 대해 말씀해 주세요."})

'깨봉선생님께서 말씀하시기를, 미적분은 크게 두 가지 주요 개념, 즉 미분과 적분으로 나뉩니다. 미분은 주어진 함수의 순간 변화율을 측정하는 과정이며, 기울기나 변화의 속도를 알아내는 데 사용됩니다. 즉, 어떤 함수가 변화할 때 그 변화의 정도를 수학적으로 표현한 것이죠. 반면, 적분은 분할된 작은 부분들의 합을 구하여 전체의 양을 찾는 과정입니다. 이는 넓이, 부피, 그리고 다른 물리적 양들을 계산하는 데 사용되죠. 따라서 미적분은 변화하는 상황을 이해하고, 그로 인해 발생하는 전체적인 결과를 예측하는 데 매우 중요한 역할을 합니다. 미적분의 이러한 개념들은 과학, 공학, 경제학 등 다양한 분야에서 광범위하게 적용되며, 복잡한 문제들을 해결하는 데 필수적인 도구로 사용됩니다.'

In [40]:
# 질문을 입력하여 전체 체인을 실행합니다.
full_chain.invoke({"question": "중력 가속도는 어떻게 계산하나요?"})

'아이작 뉴턴 선생님께서 말씀하시기를, 중력 가속도는 물체가 자유낙하할 때 받는 가속도로, 지구 표면에서의 평균값은 대략 9.81m/s²입니다. 이는 뉴턴의 만유인력 법칙을 바탕으로 계산할 수 있으며, 특정한 위치에서의 중력 가속도를 계산하려면 \\(g = \\frac{G \\cdot M}{r^2}\\) 공식을 사용합니다. 여기서 \\(G\\)는 중력 상수(약 6.674×10^-11 N(m/kg)^2), \\(M\\)은 중심 물체의 질량(예를 들어 지구의 경우 약 5.972×10^24kg), 그리고 \\(r\\)은 중심 물체의 중심으로부터의 거리입니다. 이 공식을 통해 우리는 지구나 다른 천체의 표면 또는 그 근처에서의 중력 가속도를 계산할 수 있습니다.'

In [41]:
# 질문을 입력하여 전체 체인을 실행합니다.
full_chain.invoke({"question": "RAG(Retrieval Augmented Generation)은 무엇인가요?"})

'RAG(Retrieval Augmented Generation)는 기계학습 분야에서 사용되는 기술로, 정보 검색과 생성 모델을 결합하여 특정 질문에 대한 답변을 생성합니다. 이 방법은 먼저 관련 정보를 데이터베이스에서 검색한 다음, 이 정보를 바탕으로 자연어 처리(NLP) 모델이 정확하고 관련성 높은 답변을 생성합니다.'